In [ ]:
"""
Author: Liam Bogucki
Email: lboguck@uwo.ca
First Written: Wednesday, November 13, 2024
Last Modified: Tuesday, August 12, 2025
Program Purpose: To process the AEZ data such that the map produced can be used for IAV calculations later
"""

In [ ]:
# Importing appropriate libraries.
import numpy as np
import rasterio
import matplotlib.pyplot as plt
import scipy.stats as stats
# Changing over the font
plt.rcParams['font.family'] = 'Times New Roman'

In [ ]:
#Loading in the tif file
with rasterio.open('aez_v9v2_ENSEMBLE_rcp2p6_2020s.tif') as file:
    data = file.read(1)

# The shape of the array
print(data.shape)

# Collapsing the array into the appropriate 360,720 size
array = data.reshape(360, 60, 720, 60)

#Getting mode of what the value should be in each gridcell
rescaled_array = (stats.mode(array, axis=(1,3), keepdims=False)).mode

#Verifying the shape of 360,70
print(np.shape(rescaled_array))

# These values to keep are based off of the UN AEZ numbering provided on page 153/154 of the AEZ document (See citation in paper)
# Going off of the PANGEA regions of lowland humid tropcis with no terrain limits 5, and with terrain limits 6, hihland humid tropics without terrain limits 11, and with terrain limits 12
# Also including 52 for the dominantly hydromorphic soils
# I am not including the category of land with severe soil/terrain limitations as there is no good way for me to remove these regions with the specificity given by PANGEA
rescaled_array[(rescaled_array != 5) & (rescaled_array != 6) & (rescaled_array != 11) & (rescaled_array != 12) & (rescaled_array != 52)] = 0

# Cuttting down our regions to be within the scop of the tropical lines used by PANGEA, that is, within the tropic of cancer and capricorn
# Since 0.5deg scale the tropical bounds will be 180+ 47 and 180-47...roughly
tropic_cancer = 180 - 47
tropic_capricorn = 180 +47
# Masking out the array outside the tropics by first setting to 0, and then masking anyhting that is equal to 0
rescaled_array[:tropic_cancer,:] = 0
rescaled_array[tropic_capricorn+1:,:] = 0
rescaled_array = np.ma.masked_equal(rescaled_array, 0) #masking out where 0

#Saving the plot
np.save('aez_tropical_PANGEA.npy', rescaled_array.data)



In [ ]:
#Printing what the rescaled array looks like
plt.imshow(rescaled_array)
plt.colorbar()